In [ ]:
"""
Play with OTP
"""

In [ ]:
"""
Prepare OSM data
"""

from glass.tbl.filter.osm import highways_from_osm

osm = '/mnt/d/gwork/gigs_cmcprj/gtfs/portugal-latest.osm.pbf'

lmt = '/mnt/d/gwork/gigs_cmcprj/admin/data_bgri.shp'

osm_graph = '/home/luffy/otp/coimbra.pbf'

highways_from_osm(osm, osm_graph, filter_geom=lmt, api='osmconvert')

In [ ]:
"""
OTP Service Area
"""

import os
from glass.mob.otp import otp_servarea
from glass.pys.tm import intervals_in_timerange

facilities = '/mnt/d/gwork/gigs_cmcprj/servicos/Escolas.shp'

date= '09-21-2020'

breaks = [60 * 60]
output = '/mnt/d/gwork/gigs_cmcprj/servicos/sa_otp_esc/'
bname = 'sa_esc_'

min_h = 8
max_h = 10
int_=15
time_i = intervals_in_timerange(min_h, max_h, int_)

time_str = ["{}:{}{}".format(
    str(t.hour) if t.hour < 13 else str(t.hour - 12),
    str(t.minute) if len(str(t.minute)) == 2 else '0' + str(t.minute),
    'pm' if t.hour > 12 else 'am'
) for t in time_i]

for hour in time_str:
    for brk in breaks:
        otp_servarea(
            facilities, hour, date, [brk],
            os.path.join(output, bname + hour.split(':')[0] + hour.split(':')[1] + '_' + str(int(brk/60)) + '.shp')
        )

In [ ]:
facilities = '/home/jasp/mrgis/hsp_gen.shp'

hourday='8:00am'
date= '09-21-2020'

breaks = [30 * 60]
output = '/home/jasp/mrgis/sa_smtuc_hsp_gen_8am.shp'

In [ ]:
import requests
import polyline
import pandas as pd
from shapely.geometry import LineString
from shapely.ops import linemerge
from geopandas import GeoDataFrame
import os

from glass.cons.otp import ISO_URL
from glass.dct.geo.fmshp import shp_to_obj
from glass.prj.obj import df_prj
from glass.prop.prj import get_shp_epsg
from glass.wt.shp import obj_to_shp
from glass.pys.oss import fprop
from glass.it.pd import json_obj_to_geodf
from glass.pd import merge_df

In [ ]:
facilities_df = df_prj(shp_to_obj(facilities), 4326)

get_params = [
    ('mode', 'WALK,TRANSIT'),
    ('date', date),
    ('time', hourday),
    ('maxWalkDistance', 50000)
]

breaks.sort()

for b in breaks:
    get_params.append(('cutoffSec', b))

In [ ]:
error_logs = []
# Results
# incident_id, facility_id, minutes, walkmin, tranmin, waitmin, geom
results = []

In [ ]:
for i, r in facilities_df.iterrows():
    fromPlace = str(r.geometry.y) + ',' + str(r.geometry.x)
    
    if not i:
        get_params.append(('fromPlace', fromPlace))
    else:
        get_params[-1] = ('fromPlace', fromPlace)
    
    resp = requests.get(ISO_URL, get_params, headers={'accept'  : 'application/json'})
    
    try:
        data = resp.json()
    except:
        error_logs.append([i, 'Cannot retrieve JSON Response'])
        continue
    
    gdf = json_obj_to_geodf(data, 4326)
    gdf['ffid'] = i
    
    results.append(gdf)

df_res = merge_df(results)

out_epsg = get_shp_epsg(facilities)

if out_epsg != 4326:
    df_res = df_prj(df_res, out_epsg)

obj_to_shp(df_res, "geometry", out_epsg, output)

In [ ]:
for i, r in incidents_df.iterrows():
    duration = None
    plan_record = None
    for e, _r in facilities_df.iterrows():
        fromPlace = str(r.geometry.y) + ',' + str(r.geometry.x)
        toPlace = str(_r.geometry.y) + ',' + str(_r.geometry.x)
        
        resp = requests.get(URL, params={
            'fromPlace'       : fromPlace,
            'toPlace'         : toPlace,
            'time'            : hourday,
            'date'            : date,
            'mode'            : 'TRANSIT,WALK',
            'maxWalkDistance' : 50000,
            'arriveBy'        : 'false',
            'numItineraries'  : 1
        }, headers={'accept'  : 'application/json'})
        
        data = resp.json()
        
        if "error" in data:
            error_logs.append([i, e, data["error"]])
            continue
        
        # Get Iteneraries
        it = data['plan']["itineraries"][0]
        
        # Get intenerary geometry
        geoms = [LineString(polyline.decode(p["legGeometry"]["points"], geojson=True)) for p in it['legs']]
        
        geom = linemerge(geoms)
        
        if not duration or duration > it["duration"]:
            duration = it["duration"]
            plan_record = [
                i, e, it["duration"] / 60, it["walkTime"] / 60,
                it["transitTime"] / 60, it['waitingTime'] / 60,
                geom
            ]
        
    results.append(plan_record)

In [ ]:
res_df = GeoDataFrame(pd.DataFrame(results, columns=[
    "iid", "ffid", "minutes", "walkmin", "tranmin", "waitmin", "geom"
]), crs='EPSG:4326', geometry="geom")

out_epsg = get_shp_epsg(incidents)

if out_epsg != 4326:
    res_df = df_prj(res_df, out_epsg)

obj_to_shp(res_df, "geom", out_epsg, output)

In [ ]:
"""
OTP closest facility considering relations between incidents and facilities
"""

import os
from glass.mob.otp import otp_cf_based_on_rel
from glass.pys.tm import intervals_in_timerange

incidents = '/mnt/d/gwork/gigs_cmcprj/admin/bgri_pnt.shp'
grpicol = 'idfreg'
facilities = '/mnt/d/gwork/gigs_cmcprj/servicos/ACESaude.shp'
facilities_id = 'csid'

rel_tbl = '/mnt/d/gwork/gigs_cmcprj/freg_cs.xlsx'
sheet = 'freg_cs'
group_fk = 'codfreg'
fac_fk = 'CS_FID'
day = '09-21-2020'

min_h = 8
max_h = 12
int_ = 15

time_i = intervals_in_timerange(min_h, max_h, int_)
time_str = ["{}:{}{}".format(
    str(t.hour) if t.hour < 13 else str(t.hour - 12),
    str(t.minute) if len(str(t.minute)) == 2 else '0' + str(t.minute),
    'pm' if t.hour > 12 else 'am'
) for t in time_i]

output = '/mnt/d/gwork/gigs_cmcprj/servicos/cf_otp_cs'
bname = 'cs_'

for h in time_str:
    otp_cf_based_on_rel(
        incidents, grpicol,
        facilities, facilities_id, rel_tbl, sheet, group_fk, fac_fk,
        h, day, os.path.join(output, bname + h.split(':')[0] + h.split(':')[1]  + '.shp')
    )
